# How does supervised machine learning work?
Supervised machine learning takes places in two steps - the *training* phase, and the *testing* phase.  In the training phase, you use a portion of your data to *train* your algorithm (which, in our case, is a classification algorithm).  You provide both your feature vector and your labels to the algorithm, and the algorithm searches for patterns in your data that can help associate it with a particular label.

In the testing phase, we use the classifier we trained in the previous step, and give it previously unseen feature vectors representing unseen data to the algorithm, and have the algorithm predict the label.  We can then compare the "true" label to the predicted label, and see if our classifier provides us with a good and generlizable way of accomplishing the task (in our case, the task of automatically distinguishing news sentences from romance sentences).

![imagemlsteps](images/mlsteps.png)
Source: Andrew Rosenberg


It's important to remember that we cannot use the same data we used to build the classifier to test the data; if we did, our classifier would be 100% correct all of the time!  This will not tell us how our trained classifer will perform on new, unseen data.  We therefore need to split our data into a *train set* and a *test set*.
- We will use the train set data to train our classifier
- We will use the test set data to test our classifier

First, we need to load in the Python libraries that we will be using for our analysis. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import numpy as np

### Read data in from a spreadsheet
Lets take the data we just saved out and load it back into a dataframe so that we can do some analysis with it!

In [2]:
df = pd.read_csv("df_news_romance.csv")

### Preparing data for machine learning
We're almost ready to do some machine learning!  First, we need to split our data into *feature vectors* and *labels*.  We need them separated to train the classifier.  Remember, the features we are using to train our classifier are numbers of nouns, adjectives, and adverbs are in each sentence.  (We are not using the sentences themselves as features!)

In [3]:
fv = df[["NN", "JJ"]]
fv.head()

,NN,JJ
0,11,2
1,13,2
2,16,2
3,9,3
4,5,3


In [4]:
df['label'].value_counts()

news       4623
romance    4431
Name: label, dtype: int64

We have more news sentences than romance sentences; this is not a problem, but it's something to take note of during evaluation.


### Partitioning data into train and test sets
When you are partitioning your data into train and test sets, a good place to start is to use 75% of your data for training,and 25% of your data for testing.  We want as much training data as possible, while also having enough testing data to ensure that our trained classifier is generalizable across a number of examples.  This will also lead to more accurate evalutation of our trained classifier.

Fortunately, sklearn has a function that will do exactly this!

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fv, df['label'],
                                                stratify=df['label'], 
                                                test_size=0.25,
                                                random_state = 42)

- We use the "stratify" argument because we have an uneven amount of training data; we have more news sentances than romance sentences.  By using stratify, we ensure that our classifier will take this data imbalence into account.


- In this example, we are using a fixed random state, to ensure we will always get exactly the same value when we classify.  Adding this argument is unnecessary for most types of classification; we do it here to ensure our results do not vary slightly across runs.

Let's check the size of our train and test datasets using the `.shape` attribute of train and test data. 

In [6]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6790, 2) (6790,)
(2264, 2) (2264,)


### What classifier do I use?
Chosing a classifier can be a challenging task.  However, this flowchart can give you an idea of where to start!

![algorithms_cheatsheet](../images/algorithms_cheatsheet.png)
Source: Andreas Mueller


According to this, we are going to use LinearSVC, which is a linear model for classification that separates classes using a line, a plane, or a hyperplane. SVC stands for "Support Vector Classifier", which is a type of support vector machine algorithm.


### An animated example of classification 
The following animated GIF shows an example of linear classification.

![croppedml](../images/croppedml.gif)

Source: Andrew Rosenberg

## Let's build a classification algorithm with sklearn!
One of the best things about sklearn is the simplicity of its syntax.


To do machine learning with sklearn, follow these five steps (the function names remain the same, regardless of the classifier you use!):

### Step 1:  Import your desired classifier

In [7]:
from sklearn.svm import LinearSVC

### Step 2: Create an instance of your machine learning algorithm

In [8]:
classifier = LinearSVC(random_state=42)

### Step 3:  Fit your data to your classifier (train)

In [9]:
classifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

As mentioned above, LinearSVC, which is a linear model for classification that separates classes using a line, a plane, or a hyperplane. The `classifier.fit` method searches for that line, plane, or hyperplane-which is also called the decision boundary. The dark gray line in the figure below is the decision boundary that the *LinearSVC* classifier found for this set of training data. All the data (dots) to the left of the gray line in the area with the orange background are classified as romance, while all the data to the right in the blue area are classified as news. The leftward skew of the classification space is due to the data being very dense and highly overlapping.

![LinearSVC training data](../images/training_boundary.png?)

### Step 4:  Predict labels for unseen data (test)

In [10]:
y_predict = classifier.predict(X_test)

### Step 5: Score!
Evaluate the skill of the model by computing the 
* score: how many predicted labels are the same as the actual labels 
* confusion matrix: true positive, false positive, false negative, and true negative counts

In [11]:
classifier.score(X_test, y_test)

0.70759717314487636

Right now, our classifier can predict previously unseen news and data 

In [12]:
from sklearn.metrics import confusion_matrix

In [13]:
confusion_matrix(y_test, y_predict)

array([[747, 409],
       [253, 855]])


|      |actual news | actual romance |
|:--: | :--:| :--:|
|predicted news | 747 | 409 |
|predicted romance|253 | 855|

In `LinearSVC` the `classifier.predict` decides which class a data point is in based on which side of the decision boundary, which is the gray line in the figure, the point falls on. Points in the orange area to the left of the gray line are classified as romance, while points in the blue area to the right of the gray line. Orange points in the blue area are romance texts that are misclassified as news texts, while blue points in the orange area are news texts that are misclassified as romance texts. 

![LinearSVC Testing Data](../images\testing_boundary.png?)